**CÀI ĐẶT THƯ VIỆN**

In [5]:
#Thư viện xử lý ảnh
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import cv2

#Thư viện xử lý dữ liệu
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
import numpy as np

#Thư viện tùy chỉnh mô hình
from tensorflow.keras.layers import Flatten,Dropout,Dense,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

#Thư viên đồ thị
import matplotlib.pyplot as plt
# import pickle
# import os

**ĐƯỜNG DẪN THƯ MỤC THIẾT LẬP**

In [2]:
CSV_Path='/kaggle/input/brain-tumor/Brain_11_6/_info.csv'
Image_Path='/kaggle/input/brain-tumor/Brain_11_6/'
MaxImage=5000
MyEpoch=200 # Vì dữ liệu khá lớn nên việc học nhiều giúp học được tốt từ dữ liệu đa dạng.
# Nếu tập dữ liệu nhỏ, việc chạy quá nhiều epochs có thể dẫn đến hiện tượng overfitting
# (mô hình quá tinh chỉnh cho tập dữ liệu huấn luyện mà không tổng quát hóa tốt cho dữ liệu mới).
MyBatch=16 # Tập dữ liệu khá lớn nên cần chọn bath size lớn tùy vào tập dữ liệu và bộ nhớ thiết bị
# Một số nghiên cứu đã chỉ ra rằng batch size lớn có thể đạt được độ chính xác tốt hơn
# và ổn định hơn trong quá trình huấn luyện.

**ĐỌC ẢNH VÀ TIỀN XỬ LÝ ẢNH**

In [ ]:
Dataset=pd.read_csv(CSV_Path)
filenames=Dataset.iloc[:MaxImage,:-5].values
labels=Dataset.iloc[:MaxImage,1:2].values
bboxes=Dataset.iloc[:MaxImage,2:].values

# Xóa biến không xử dụng nữa để giải phóng vùng nhớ
del Dataset

data = []
for filename in filenames:
  imagePath=Image_Path+filename[0]
  # Đọc ảnh và đưa ảnh về kích thước 224x224
  image=load_img(imagePath,target_size=(224, 224))
  # Chuyển ảnh sang dạng mảng(ma trận) sau đó đưa vào biến data giữ
  image = img_to_array(image)
  data.append(image)

In [6]:
# Vì ma trận ảnh về kiểu dữ liệu float32 trong khoản 0-1 để có hiệu xuất tốt hơn với 1 số lý do:
# *Tính toán dễ dàng chính xác hơn, các phép tính tích chập, gradient được tối ưu.
# *Giúp mô hình dễ dàng học thông tin quan trọng. Trong khi ma trận cần phải được chuẩn hóa lại.
# *Tối ưu hóa phần cứng vì 1 số GPU tính toán tốt trên kiểu dữ liệu float32
data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)
# Do đưa ảnh về float32 và vì toàn bộ ảnh đều có kích thước 640x640 nên có thể chia trực tiếp
# trên mảng để lấy ra tọa độ box trên ma trận ảnh lúc này là kiểu float32
bboxes = np.array(bboxes, dtype="float32") / 640.0
# Vì label là chữ nên cần mã hóa thành dạng(one-hot vector) để dễ xử lý hơn
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

**TÁCH TẬP DỮ LIỆU**

In [7]:
# Tách tập dữ liệu ra thành tập dữ liệu train(80%) và val(20%)
# Tập val được sử dụng để đánh giá hiệu suất của mô hình trên dữ liệu không được sử dụng
# trong quá trình huấn luyện, giúp đánh giá mức độ tổng quát hóa của mô hình.
split = train_test_split(data, labels, bboxes,test_size=0.20, random_state=42)
(trainImages, valImages) = split[:2]
(trainLabels, valLabels) = split[2:4]
(trainBBoxes, valBBoxes) = split[4:]

#Xóa biến không xử dụng nữa để giải phó ng vùng nhớ
del data,bboxes,filenames,split

**CÀI ĐẶT THƯ VIỆN VÀ THIẾT LẬP MODEL**

In [9]:
from tensorflow.keras.applications import ResNet101
Aplication_Name='ResNet101'

In [ ]:
# Tạo model input thuộc loại phân loại hình ảnh weights="imagenet"
# include_top=False loại bỏ layer đầu
# Đầu vào của model có kích thước 224x224 3(số lớp => ảnh màu)
Model_input = ResNet101(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))
# Tắt các trọng số của Model_Input
Model_input.trainable = False
flatten = Model_input.output
# Flatten(làm phẳng) biến đổi đầu vào từ 3D thành 1D
flatten = Flatten()(flatten)

**THAY ĐỔI KÍCH THƯỚC ĐẦU RA**

In [11]:
# relu mô phỏng những nơ ron có tỷ lệ truyền xung qua axon
# sigmoid sự thay đổi nhỏ trong input dẫn đến một kết quả output ko mấy thay đổi.

# Lấy thông tin của ảnh khi được truyền vào
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
# Lấy thông tin của ảnh khi được truyền vào xử dụng để đoán tọa độ box
bboxHead = Dense(4, activation="sigmoid",name="bounding_box")(bboxHead)

# Lấy thông tin của ảnh khi được truyền vào(flatten)
softmaxHead = Dense(512, activation="relu")(flatten)
# Lớp này giúp ngăn chặn việc overfitting bằng cách tắt ngẫu nhiên 50% đơn vị(512)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.5)(softmaxHead)
#Lấy thông tin của ảnh khi được truyền vào xử dụng để đoán nhãn của đối tượng
softmaxHead = Dense(len(lb.classes_), activation="softmax",name="class_label")(softmaxHead)

model = Model(
    inputs=Model_input.input,
    outputs=(bboxHead, softmaxHead))

In [12]:
losses = {
    #Định nghĩa "categorical_crossentropy" thích hợp cho dữ liệu dạng one-hot vector
    "class_label": "categorical_crossentropy",
    #Định nghĩa "mean_squared_error" do lường sai số trung bình của tọa độ box
    "bounding_box": "mean_squared_error",
}
#Định nghĩa độ quan trọng của loss
lossWeights = {
    "class_label": 1.0,
    "bounding_box": 1.0
}

**THIẾT LẬP TÙY CHỌN CẢI TIẾN CHO MODEL**

In [13]:
#Điều chỉnh tốc độ học của mô hình với, Adam tự động điều chỉnh mức độ học tùy thuộc
#vào sự thay đổi của từng tham số trong quá trình huấn luyện
opt = Adam(learning_rate=0.0001)
#Cấu hình mô hình để sử dụng các thông số cần thiết cho quá trình huấn luyện.
model.compile(loss=losses, optimizer=opt, metrics=["accuracy"], loss_weights=lossWeights)

In [14]:
#Định nghĩa lại cho trainTargets và valTargets
trainTargets = {
    "class_label": trainLabels,
    "bounding_box": trainBBoxes
}
valTargets = {
    "class_label": valLabels,
    "bounding_box": valBBoxes
}

**HUẤN LUYỆN MODEL**

In [ ]:
H = model.fit(
    trainImages, trainTargets,
    validation_data=(valImages, valTargets),
    batch_size=MyBatch,
    epochs=MyEpoch,
    verbose=1)
#verbose hiển thị thông tin huấn luyện chi tiết của mỗi epoch

**LƯU MODEL**

In [16]:
model.save('/kaggle/working/'+Aplication_Name+'.h5')

**ĐỒ THỊ ACCURACY VÀ LOSS CỦA MODEL**

In [ ]:
N = np.arange(0, 200)
plt.style.use("ggplot")
(fig, ax) = plt.subplots(1, 2, figsize=(12, 2))

acc=H.history["class_label_accuracy"]
val_acc=H.history["val_class_label_accuracy"]

ax[0].plot(N,acc,label="label_accuracy",color='darkblue')
ax[0].plot(N,val_acc,label="val_label_accuracy",color='cyan')
ax[0].set_xlabel("Epoch")
ax[0].set_ylabel("Accuracy")
ax[0].legend()

loss=H.history["class_label_loss"]
val_loss=H.history["val_class_label_loss"]

ax[1].plot(N,loss,label="label_loss",color='crimson')
ax[1].plot(N,val_loss,label="val_label_loss",color='salmon')
ax[1].set_xlabel("Epoch")
ax[1].set_ylabel("Loss")
ax[1].legend()

**In các thông số kiểm thử sau huấn luyện**

In [ ]:
# from tensorflow.keras.models import load_model
# model=load_model('/kaggle/input/my-model/ResNet101 (1).h5')

#Lấy thông tin kiểm thử của mô hình
results=model.evaluate(trainImages, trainTargets,verbose=0)
print(f"Total Loss: {results[0]}")
print(f"Bounding Box Loss: {results[1]}")
print(f"Class Label Loss: {results[2]}")
print(f"Bounding Box Accuracy: {results[3]}")
print(f"Class Label Accuracy: {results[4]}")

In [18]:
del trainLabels, testLabels
del trainBBoxes, testBBoxes

**DỰ ĐOÁN ĐỂ HIỂN THỊ MA TRẬN NHẰM LẪN**

In [ ]:
(boxPreds, labelPreds) = model.predict(trainImages)

In [26]:
labelPreds=(np.rint(labelPreds)).astype(int)
labelPreds=np.array(labelPreds, dtype="float32")
labelPreds=np.argmax(labelPreds,axis=1)
labelTargets=trainTargets['class_label']
labelTargets=np.argmax(labelTargets,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
confusion_mtx = confusion_matrix(labelTargets, labelPreds)
confusion_mtx_percent = confusion_mtx.astype('float') / confusion_mtx.sum(axis=1)[:, np.newaxis]
f,ax = plt.subplots(figsize=(8,5))
sns.heatmap(confusion_mtx_percent, annot=True, fmt=".2%",cmap='Blues')
ax.xaxis.set_ticklabels(['normal', 'tumor']);
ax.yaxis.set_ticklabels(['normal', 'tumor']);

**HIỂN THỊ ĐỒ THỊ ROC-AUC**

In [ ]:
ROC_AUC_Path='/kaggle/working/'+Aplication_Name+'.png'

from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
import matplotlib.pyplot as plt
from itertools import cycle

n_classes = 2

# Khai báo biến testImages và testTargets
testImages = []
testTargets = []

# Dự đoán trên tập validation
(temp,y_score) = model.predict(testImages)

# Binarize the output
y_test_bin = label_binarize(testTargets['class_label'], classes=[0, 1])

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Plot all ROC curves
plt.figure()
plt.plot(fpr["micro"], tpr["micro"],
         label='Đường cong trung bình ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]))

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color,
             label='ROC curve của lớp {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Đường cong dự đoán của não trên 2 lớp')
plt.legend(loc="lower right")
plt.savefig(ROC_AUC_Path)
plt.show()

**DỰ ĐOÁN ẢNH**

In [27]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import cv2
import numpy as np
import imutils
import matplotlib.pyplot as plt

model = load_model('path')


image_files = ['path_image1', 
               'path_image2', 
               'path_image3']

plt.figure(figsize=(30, 10))  # Adjust as needed

for idx, image_file in enumerate(image_files):
    image = load_img(image_file, target_size=(224, 224))
    image = img_to_array(image) / 255.0
    image = np.expand_dims(image, axis=0)
    (boxPreds, labelPreds) = model.predict(image)
    i = np.argmax(labelPreds, axis=1)
    label = 'tumor' if i else 'normal'
    
    size_label = "N/A"
    if label=='tumor':
        (startX, startY, endX, endY) = boxPreds[0]
        image = cv2.imread(image_file)
        image = imutils.resize(image, width=600)
        (h, w) = image.shape[:2]
        startX = int(startX * w)
        startY = int(startY * h)
        endX = int(endX * w)
        endY = int(endY * h)
        y = startY - 10 if startY - 10 > 10 else startY + 10
        
        # Compute tumor size
        tumor_width = endX - startX
        tumor_height = endY - startY
        
        # Decide the unit based on the size
        size_label = f"{tumor_width}mm x {tumor_height}mm"
        if tumor_width >= 10 and tumor_height >= 10:
            tumor_width /= 10
            tumor_height /= 10
            size_label = f"{tumor_width}mm x {tumor_height}mm"
        
        cv2.putText(image, f"{label}", (startX, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (0, 0, 255), 2)
        cv2.rectangle(image, (startX, startY), (endX, endY),(0, 0, 255), 2)
    else:
        image = cv2.imread(image_file)
        image = imutils.resize(image, width=600)
        cv2.putText(image, label, (20,20), cv2.FONT_HERSHEY_SIMPLEX,0.7, (0, 255, 0), 2)
    
    plt.subplot(1, len(image_files), idx+1)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(f'{label}: {np.max(labelPreds) * 100:.2f}%, size: {size_label}')
    plt.axis('off')
plt.show()